# 第19章 马尔可夫链蒙特卡罗法 之 核心思想

    19.3 马尔可夫链蒙特卡罗法    
        19.3.1 基本想法    
        19.3.2 基本步骤    
        19.3.3 马尔可夫链蒙特卡罗法与统计学习    
        

参考资料

1. [TowardsDataScience - Markov Chain Monte Carlo in Python](https://towardsdatascience.com/markov-chain-monte-carlo-in-python-44f7e609be98)

2. [TowardsDataScience - Markov Chain Monte Carlo in Python (GitHub Repo)](https://github.com/WillKoehrsen/ai-projects/blob/master/markov_chain_monte_carlo/markov_chain_monte_carlo.ipynb)

3. [An Introduction to MCMC for Machine Learning](https://www.cs.princeton.edu/courses/archive/spr06/cos598C/papers/AndrieuFreitasDoucetJordan2003.pdf)

4. [PRML Sampling Methods (GitHub Repo)](https://github.com/ctgk/PRML/blob/master/notebooks/ch11_Sampling_Methods.ipynb)

In [1]:
import random
import math
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

%matplotlib inline
sns.set_style('darkgrid')
plt.rcParams['figure.figsize'] = (12, 8)

马尔可夫链蒙特卡罗法（Markov Chain Monte Carlo, MCMC）,则是以马尔可夫链（Markov chain）为概率模型的蒙特卡罗法。
**马尔可夫链蒙特卡罗法构建一个马尔可夫链，使其平稳分布就是要进行抽样的分布，首先基于该马尔可夫链进行随机游走，产生样本的序列，之后使用该平稳分布的样本进行近似数值计算。**

马尔可夫链蒙特卡罗法被应用于概率分布的估计、定积分的近似计算、最优化问题的近似求解等问题，特别是被应用于统计学习中概率模型的学习与推理，是重要的统计学习计算方法。

The trouble with these methods is that they do not work well in high dimensional spaces. The most popular method for sampling from high-dimensional distributions is Markov chain Monte Carloor MCMC.

## 19.3 马尔可夫链蒙特卡罗法    

###     19.3.1 基本想法    

马尔可夫链蒙特卡罗法更适合于随机变量是多元的、密度函数是非标准形式的、随机变量各分量不独立等情况。

假设多元随机变量$x$，满足$x \in \mathcal{X}$，其概率密度函数为 $p(x)$，$f(x)$ 为定义在$x \in \mathcal{X}$上的函数，目标是获得概率分布 $p(x)$ 的样本集合，以及求函 数 $f(x)$ 的数学期望 $E_{p(x)}[f(x)]$。

#### 马尔可夫链蒙特卡罗法的基本想法

在随机变量$x$的状态空间$S$上定义一个满足遍历定理的马尔可夫链 $X = \{ X_0, X_1, \cdots, X_t, \cdots \}$ ，使其平稳分布就是抽样的目标分布 $p(x)$。
然后在这个马尔可夫链上进行随机游走，每个时刻得到一个样本。

根据遍历定理，当时间趋于无穷时，样本的分布趋近平稳分布，样本的函数均值趋近函数的数学期望。
所以，当时间足够长时（时刻大于某个正整数 $m$）,在之后的时间（时刻小于等于某个正整数$n$ , $n > m$）里随机游走得到的样本集合 $\{x_{m+1},x_{m+2},\cdots, x_n \}$ 就 是目标概率分布的抽样结果，得到的函数均值（遍历均值）就是要计算的数学期望值：

$$ \hat{E} f=\frac{1}{n-m} \sum_{i=m+1}^{n} f\left(x_{i}\right) $$

到时刻 $m$ 为止的时间段称为**燃烧期**。

**如何构建具体的马尔可夫链成为这个方法的关键**

连续变量的时候，需要定义转移核函数；
离散变量的时候，需要定义转移矩阵。
一个方法是定义特殊的转移核函数或者转移矩阵，构建可逆马尔可夫链，这样可以保证遍历定理成立。
常用的马尔可夫链蒙特卡罗法有 **Metropolis-Hastings算法**、**吉布斯抽样**。

由于这个马尔可夫链满足遍历定理，随机游走的起始点并不影响得到的结果，即从不同的起始点出发，都会收敛到同一平稳分布。

马尔可夫链蒙特卡罗法的收敛性的判断通常是经验性的。
比如，在马尔可夫链上进行随机游走，检验遍历均值是否收敛。
具体地，每隔一段时间取一次样本，得到多个样本以后，计算遍历均值，当计筇的均值稳定 后，认为马尔可夫链己经收敛。
再比如，在马尔可夫链上并行进行多个随机游走，比较各个随机游走的遍历均值是否接近一致。

马尔可夫链蒙特卡罗法中得到的样本序列，相邻的样本点是相关的，而不是独立的。
因此，在需要独立样本时，可以在该样本序列中再次进行随机抽样，比如每隔一段时间取一次样本，将这样得到的子样本集合作为独立样本集合。

马尔可夫链蒙特卡罗法比**接受-拒绝法**更容易实现，因为只需要定义马尔可夫链，而不需要定义建议分布。
一般来说马尔可夫链蒙特卡罗法比**接受-拒绝法**效率更高，没有大量被拒绝的样本，虽然**燃烧期**的样本也要抛弃。

#### 几个重要问题
* 1. 如何定义马尔可夫链，保证马尔可夫链蒙特卡罗法的条件成立.
* 2. 如何确定收敛步数 $m$，保证样本抽样的无偏性。
* 3. 如何确定迭代步数 $n$，保证遍历均值计算的精度。

###     19.3.2 基本步骤    

* 1. 在随机变量 $x$ 的状态空间 $\mathcal{S}$ 上构造一个满足遍历定理的马尔可夫链，使其平稳分布为目标分布 $p(x)$;
* 2. 从状态空间的某一点 $x_0$ 出发，用构造的马尔可夫链进行随机游走，产生样本序列 $x_0, x_1, \cdots, x_t, \cdots $；
* 3. 应用马尔可夫链的遍历定理，确定正整数 $m$ 和 $n$，$(m<n)$， 得到样本集合 $\{x_{m+1},x_{m+2},\cdots, x_n \}$，求得函数 $f(x)$ 的均值（遍历均值）

$$ \hat{E} f=\frac{1}{n-m} \sum_{i=m+1}^{n} f\left(x_{i}\right) $$

###     19.3.3 马尔可夫链蒙特卡罗法与统计学习    

#### MCMC的可视化案例

1. [Random Walk Metropolis Hastings](https://chi-feng.github.io/mcmc-demo/app.html?algorithm=RandomWalkMH&target=banana)
2. [Adaptive Metropolis Hastings](https://chi-feng.github.io/mcmc-demo/app.html?algorithm=AdaptiveMH&target=banana) [1]
3. [Hamiltonian Monte Carlo](https://chi-feng.github.io/mcmc-demo/app.html?algorithm=HamiltonianMC&target=banana) [2]
4. [No-U-Turn Sampler](https://chi-feng.github.io/mcmc-demo/app.html?algorithm=NaiveNUTS&target=banana) [2]
5. [Metropolis-adjusted Langevin Algorithm (MALA)](https://chi-feng.github.io/mcmc-demo/app.html?algorithm=MALA&target=banana) [3]
6. [Hessian-Hamiltonian Monte Carlo (H2MC)](https://chi-feng.github.io/mcmc-demo/app.html?algorithm=H2MC&target=banana) [4]
7. [Gibbs Sampling](https://chi-feng.github.io/mcmc-demo/app.html?algorithm=GibbsSampling&target=banana)
8. [Stein Variational Gradient Descent (SVGD)](https://chi-feng.github.io/mcmc-demo/app.html?algorithm=SVGD&target=banana&delay=0) [5]
9. [Nested Sampling with RadFriends (RadFriends-NS)](https://chi-feng.github.io/mcmc-demo/app.html?algorithm=RadFriends-NS&target=banana) [6]

项目地址在 https://github.com/chi-feng/mcmc-demo.

参考文献

1. H. Haario, E. Saksman, and J. Tamminen, [An adaptive Metropolis algorithm](http://projecteuclid.org/euclid.bj/1080222083) (2001)
2. M. D. Hoffman, A. Gelman, [The No-U-Turn Sampler: Adaptively Setting Path Lengths in Hamiltonian Monte Carlo](http://arxiv.org/abs/1111.4246) (2011)
3. G. O. Roberts, R. L. Tweedie, [Exponential Convergence of Langevin Distributions and Their Discrete Approximations](http://www2.stat.duke.edu/~scs/Courses/Stat376/Papers/Langevin/RobertsTweedieBernoulli1996.pdf) (1996)
4. Li, Tzu-Mao, et al. [Anisotropic Gaussian mutations for metropolis light transport through Hessian-Hamiltonian dynamics](https://people.csail.mit.edu/tzumao/h2mc/) ACM Transactions on Graphics 34.6 (2015): 209.
5. Q. Liu, et al. [Stein Variational Gradient Descent: A General Purpose Bayesian Inference Algorithm](http://www.cs.dartmouth.edu/~dartml/project.html?p=vgd) Advances in Neural Information Processing Systems. 2016.
6. J. Buchner [A statistical test for Nested Sampling algorithms](https://arxiv.org/abs/1407.5459) Statistics and Computing. 2014.
